In [21]:
import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
import shutil


In [6]:
directory = directory_project
update_plate_info(directory)

In [22]:
directory = directory_scratch+'/data/'
update_plate_info(directory)

In [16]:
directory = directory_scratch+'/data/'
listdir = os.listdir(directory)
listdir

['20211013_0800_Plate39',
 '20211015_0758_Plate39',
 '20211016_0954_Plate37',
 '20211013_0830_Plate33',
 '20211014_0843_Plate39',
 '20211015_0751_Plate37',
 '20211007_1541_Plate17',
 '20211014_0836_Plate37',
 '20211012_0851_Plate37',
 '20211013_0856_Plate37',
 '20211012_0848_Plate33',
 '20211016_0939_Plate33',
 '20211014_0821_Plate33',
 '20211016_1002_Plate39',
 '20211012_0827_Plate39',
 '20211015_0736_Plate33']

In [42]:
im = imageio.imread('/scratch-shared/amftrack/data/Img_r01_c05.tif')
for dire in listdir:
    for x in range(1,11):
        for y in range(1,16):
            strix= str(x) if x>=10 else f'0{x}'
            striy= str(y) if y>=10 else f'0{y}'
            path = f'{directory}/{dire}/Img/Img_r{strix}_c{striy}.tif'
            print(striy,path,os.path.getsize(path))
            if not os.path.isfile(path):
                f = open(path, 'w')
            if os.path.getsize(path)==0:
                imageio.imwrite(path,im*0)
                # f = open(path, 'w')
                # print(path,os.path.isfile(path))

01 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c01.tif 12288304
02 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c02.tif 12288304
03 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c03.tif 12288304
04 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c04.tif 12288304
05 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c05.tif 12288387
06 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c06.tif 12288387
07 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c07.tif 12288387
08 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c08.tif 12288387
09 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c09.tif 12288387
10 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c10.tif 12288387
11 /scratch-shared/amftrack//data//20211013_0800_Plate39/Img/Img_r01_c11.tif 12288387
12 /scratch-shared/amftrack//data//20211013_0800_Plate

In [34]:
im = imageio.imread('/scratch-shared/amftrack/data/Img_r01_c05.tif')
im*0


Array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [31]:
for dire in listdir:
    for x in range(1,11):
        for y in range(1,16):
            strix= str(x) if x>=10 else f'0{x}'
            striy= str(y) if x>=10 else f'0{y}'
            path = f'{directory}/{dire}/Img/Img_r{strix}_c{striy}.tif'
            print(os.path.getsize(path))
            if not os.path.isfile(path):
                f = open(path, 'w')
                print(path,os.path.isfile(path))

0
0
0
0
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
0
0
0
12288387
12288387
12288387
12288387
12288387
12288387
0
0
0
0
0
0
0
0
12288387
12288387
12288387


In [ ]:
for dire in listdir:
    f = open(f'{directory}/{dire}/param.m', 'w')

In [5]:
for dire in listdir:
    f = open(f'{directory}/{dire}/param.m', 'w')

In [26]:
plates = [22,10,76,26,152,102,59,94,69,91,80]
folders = all_folders.loc[np.invert(all_folders['Plate'].isin(plates))]
folders
# folders = all_folders.loc[all_folders['Plate']==plate*all_folders['/Analysis/skeleton_realigned_compressed.mat']]

folder Plate PrincePos    root strain    medium split  \
0     20201218_2336_Plate26    42        26  Carrot     A5  001P100N     Y   
1     20201206_0446_Plate24    60        24  Carrot     A5  001P100N     Y   
2     20201209_1146_Plate30    67        30  Carrot     A5  001P100N     Y   
3     20201230_0719_Plate11   145        11  Carrot     A5  001P100N     Y   
4     20201219_1945_Plate29    38        29  Carrot     A5  001P100N     Y   
...                     ...   ...       ...     ...    ...       ...   ...   
1584  20201127_2159_Plate35   134        35  Carrot     A5  001P100N     Y   
1590  20201128_2159_Plate35   134        35  Carrot     A5  001P100N     Y   
1594  20201128_1759_Plate35   134        35  Carrot     A5  001P100N     Y   
1602  20201128_0959_Plate35   134        35  Carrot     A5  001P100N     Y   
1615  20201129_0159_Plate35   134        35  Carrot     A5  001P100N     Y   

        Temp CrossDate /Img/TileConfiguration.txt.registered  ...  \
0     25to30  20201204                                  True  ...   
1     25to30  20201204                                  True  ...   
2     25to30  20201202                                  True  ...   
3         20  20201214                                 False  ...   
4     25to30  20201202                                  True  ...   
...      ...       ...                                   ...  ...   
1584     NaN  20201127                                 False  ...   
1590     NaN  20201127                                 False  ...   
1594     NaN  20201127                                 False  ...   
1602     NaN  20201127                                 False  ...   
1615     NaN  20201127                                 False  ...   

     /Analysis/skeleton_masked_compressed.mat  \
0                                       False   
1                                       False   
2                                       False   
3                                       False   
4                                       False   
...                                       ...   
1584                                    False   
1590                                    False   
1594                                    False   
1602                                    False   
1615                                    False   

     /Analysis/skeleton_pruned_compressed.mat /Analysis/transform.mat  \
0                                       False                   False   
1                                       False                   False   
2                                       False                   False   
3                                       False                   False   
4                                       False                   False   
...                                       ...                     ...   
1584                                    False                   False   
1590                                    False                   False   
1594                                    False                   False   
1602                                    False                   False   
1615                                    False                   False   

     /Analysis/transform_corrupt.mat  \
0                              False   
1                              False   
2                              False   
3                              False   
4                              False   
...                              ...   
1584                           False   
1590                           False   
1594                           False   
1602                           False   
1615                           False   

     /Analysis/skeleton_realigned_compressed.mat /Analysis/nx_graph_pruned.p  \
0                                          False                       False   
1                                          False                       False   
2                                          False                    

In [25]:
folders = folders.loc[all_folders['Plate']==102]
folders

Empty DataFrame
Columns: [folder, Plate, PrincePos, root, strain, medium, split, Temp, CrossDate, /Img/TileConfiguration.txt.registered, /Analysis/skeleton_compressed.mat, /Analysis/skeleton_masked_compressed.mat, /Analysis/skeleton_pruned_compressed.mat, /Analysis/transform.mat, /Analysis/transform_corrupt.mat, /Analysis/skeleton_realigned_compressed.mat, /Analysis/nx_graph_pruned.p, /Analysis/nx_graph_pruned_width.p, /Analysis/nx_graph_pruned_labeled.p, date, Pbait]
Index: []

[0 rows x 21 columns]

In [27]:
for index, row in folders.iterrows():
    path = f'{directory}{row["folder"]}'
    print(path)
    shutil.rmtree(path)

/projects/0/einf914/data/20201218_2336_Plate26
/projects/0/einf914/data/20201206_0446_Plate24
/projects/0/einf914/data/20201209_1146_Plate30
/projects/0/einf914/data/20201230_0719_Plate11
/projects/0/einf914/data/20201219_1945_Plate29
/projects/0/einf914/data/20210103_1519_Plate11
/projects/0/einf914/data/20201219_1314_Plate12
/projects/0/einf914/data/20201210_1114_Plate26
/projects/0/einf914/data/20201204_1646_Plate24
/projects/0/einf914/data/20201228_1120_Plate11
/projects/0/einf914/data/20201206_1653_Plate26
/projects/0/einf914/data/20201221_1925_Plate22
/projects/0/einf914/data/20201226_1118_Plate11
/projects/0/einf914/data/20201208_1513_Plate29
/projects/0/einf914/data/20201223_0329_Plate24
/projects/0/einf914/data/20201205_0503_Plate29
/projects/0/einf914/data/20201221_1156_Plate38
/projects/0/einf914/data/20210104_0322_Plate12
/projects/0/einf914/data/20201217_0722_Plate26
/projects/0/einf914/data/20201216_0316_Plate24
/projects/0/einf914/data/20201205_0453_Plate26
/projects/0/e

In [2]:
plate_numbers = [344]
for plate_number in plate_numbers:
    plate = get_postion_number(plate_number)
    print(plate)
    directory = directory_project
    listdir = os.listdir(directory)
    list_dir_interest = [name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
    dates_datetime = get_dates_datetime(directory,plate)
    print(0,plate_number)
    for i in range(len(list_dir_interest)):
        new_plate_number = get_plate_number(plate,dates_datetime[i])
        directory_name = get_dirname(dates_datetime[i],plate)
        path_snap=directory+directory_name
    #     print(path_snap)
        if plate_number == new_plate_number:
            shutil.rmtree(path_snap)


10
0 344
